### 교차 검증
- 부족한 데이터셋 및 특정 데이터에 과대적한되는 문제 해결하기 위한 방안
- 학습 데이터셋을 일정 크기의 데이터로 n개 분리 후 1/n은 검증용, 나머지는 학습용으로 사용

[1] 모듈 로딩 <hr>

In [193]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

x = np.array([[1, 2],[3,4],[1, 2],[3,4]])
y = np.array([1,2,3,4])


In [194]:
# KFold 인스턴스 생성 => 데이터를 2개로 분할해주는 객체
k_fold = KFold(n_splits=2)

In [195]:
# 데이터 분할
datasets = k_fold.split(x) # 반환값 : ndarray

In [196]:
for train,test in datasets:
    print(train,test)

[2 3] [0 1]
[0 1] [2 3]


In [197]:
### perch.csv 파일 데이터 기본 5등분
perchDF = pd.read_csv('../data/perch3.csv')
perchDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Weight  56 non-null     float64
 1   Length  56 non-null     float64
 2   Height  56 non-null     float64
 3   Width   56 non-null     float64
dtypes: float64(4)
memory usage: 1.9 KB


In [198]:
# perchDF => 5등분
fold_5 = KFold(n_splits=5)
#n_splits = k 
# train = k-1/k
# test = 1/k

datasets = fold_5.split(perchDF) # 비율을 맞춰야 함 => fold.split(x,y) -> y 추가

In [199]:
for idx,(train,test) in enumerate(datasets):   # 튜플로 반환되는 값은 받을때도 튜플에다 지정해줘야 언패킹 가능
    print(f'{idx+1} => {train.shape, test.shape}')  # 총 5등분으로 나누어진 것 확인 가능
    print(train, test, sep ='\n') # 행 번호

1 => ((44,), (12,))
[12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]
[ 0  1  2  3  4  5  6  7  8  9 10 11]
2 => ((45,), (11,))
[ 0  1  2  3  4  5  6  7  8  9 10 11 23 24 25 26 27 28 29 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]
[12 13 14 15 16 17 18 19 20 21 22]
3 => ((45,), (11,))
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]
[23 24 25 26 27 28 29 30 31 32 33]
4 => ((45,), (11,))
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 45 46 47 48 49 50 51 52 53 54 55]
[34 35 36 37 38 39 40 41 42 43 44]
5 => ((45,), (11,))
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
[45 46 47 48 49 50 51 52 53 54 55]


In [200]:
irisDF = pd.read_csv('../data/iris.csv')
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [201]:
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF['variety']

In [202]:
k_fold = KFold(n_splits=3,shuffle = True )
ret_k = k_fold.split(featureDF,targetSR) # generator는 한번 순회하면 못쓰는 타입

In [217]:
ret_k = k_fold.split(featureDF)
for idx,(train,test) in enumerate(ret_k):
    print(f'================{idx+1}=================') 
    print(targetSR[train].value_counts())

================1=================
variety
Virginica     35
Versicolor    33
Setosa        32
Name: count, dtype: int64
================2=================
variety
Setosa        34
Virginica     34
Versicolor    32
Name: count, dtype: int64
================3=================
variety
Versicolor    35
Setosa        34
Virginica     31
Name: count, dtype: int64


In [215]:
ret_k = k_fold.split(featureDF,targetSR)
trainList = []
testList = []
for idx,(train,test) in enumerate(ret_k):
    # train = train.tolist()  # 학습용 데이터 인덱스
    # test = test.tolist()  # 테스트용 데이터 인덱스
    
    # 비율 확인
    print(targetSR[train].value_counts()/len(train))
    print(targetSR[test].value_counts()/len(test))
    
    xtrain = featureDF.loc[train,:]
    ytrain = targetSR[train]  # 수정
    
    xtest = featureDF.loc[test,:]  # 수정
    ytest = targetSR[test]
    
    # 분류 모델 학습
    
    model = LogisticRegression(max_iter=3000, solver = 'liblinear') # 파라미터 for문 돌려서 튜닝
    model.fit(xtrain, ytrain)
    
    # 훈련 및 검증용 성능
    train_score = model.score(xtrain, ytrain)
    test_score = model.score(xtest, ytest)
    print(train_score)
    
    # 예측
    pre = model.predict(xtest)
    
    trainList.append(train_score)
    testList.append(test_score)

variety
Virginica     0.35
Setosa        0.33
Versicolor    0.32
Name: count, dtype: float64
variety
Versicolor    0.36
Setosa        0.34
Virginica     0.30
Name: count, dtype: float64
0.95
variety
Virginica     0.37
Versicolor    0.32
Setosa        0.31
Name: count, dtype: float64
variety
Setosa        0.38
Versicolor    0.36
Virginica     0.26
Name: count, dtype: float64
0.96
variety
Setosa        0.36
Versicolor    0.36
Virginica     0.28
Name: count, dtype: float64
variety
Virginica     0.44
Setosa        0.28
Versicolor    0.28
Name: count, dtype: float64
0.95


In [216]:
sum(trainList)/len(trainList)

0.9533333333333333

In [247]:
trainList = []
testList = []

ret_k = k_fold.split(featureDF,targetSR)
for idx,(train,test) in enumerate(ret_k):
    train = train.tolist()  # 학습용 데이터 인덱스
    test = test.tolist()  # 테스트용 데이터 인덱스
    
    # 비율 확인
    print(targetSR[train].value_counts()/len(train))
    print(targetSR[test].value_counts()/len(test))
    
    xtrain = featureDF.loc[train,:]
    ytrain = targetSR[train]  # 수정
    
    xtest = featureDF.loc[test,:]  # 수정
    ytest = targetSR[test]
    
    # 분류 모델 학습
    
    model = LogisticRegression(max_iter=3000,solver='saga', penalty=p_name) # 파라미터 for문 돌려서 튜닝
    model.fit(xtrain, ytrain)
    
    # 예측
    
    # 훈련 및 검증용 성능
    train_score = model.score(xtrain, ytrain)
    test_score = model.score(xtest, ytest)
    print(train_score)
    trainList.append(train_score)
    testList.append(test_score)

variety
Setosa        0.38
Virginica     0.34
Versicolor    0.28
Name: count, dtype: float64
variety
Versicolor    0.44
Virginica     0.32
Setosa        0.24
Name: count, dtype: float64
0.97
variety
Virginica     0.39
Versicolor    0.37
Setosa        0.24
Name: count, dtype: float64
variety
Setosa        0.52
Versicolor    0.26
Virginica     0.22
Name: count, dtype: float64
0.98
variety
Setosa        0.38
Versicolor    0.35
Virginica     0.27
Name: count, dtype: float64
variety
Virginica     0.46
Versicolor    0.30
Setosa        0.24
Name: count, dtype: float64
0.97
variety
Setosa        0.34
Virginica     0.34
Versicolor    0.32
Name: count, dtype: float64
variety
Versicolor    0.36
Setosa        0.32
Virginica     0.32
Name: count, dtype: float64
0.97
variety
Virginica     0.36
Versicolor    0.35
Setosa        0.29
Name: count, dtype: float64
variety
Setosa        0.42
Versicolor    0.30
Virginica     0.28
Name: count, dtype: float64


C:\Users\kdp\.conda\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kdp\.conda\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kdp\.conda\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.99
variety
Setosa        0.37
Versicolor    0.33
Virginica     0.30
Name: count, dtype: float64
variety
Virginica     0.40
Versicolor    0.34
Setosa        0.26
Name: count, dtype: float64
0.97
variety
Virginica     0.40
Setosa        0.33
Versicolor    0.27
Name: count, dtype: float64
variety
Versicolor    0.46
Setosa        0.34
Virginica     0.20
Name: count, dtype: float64


ValueError: l1_ratio must be specified when penalty is elasticnet.